## DICE FUNCTION TEST

In [107]:
import torch
import numpy as np
from torchmetrics.functional.classification import dice

In [108]:
def dice_coefficient(pred: torch.Tensor, target: torch.Tensor, epsilon: float = 1e-6) -> float:
    """
    Calculate the Dice coefficient between predictions and target.
    This version assumes that predictions are probabilities and targets are binary masks.
    """

    if pred.dim() == 4 and pred.shape[1] == 1:  
        pred = torch.softmax(pred,dim=1)
    else:
        pred = torch.sigmoid(pred)
    
    pred_binary = (pred > 0.5).float()    
    pred_binary = pred_binary.contiguous().view(-1)
    
    target = target.contiguous().view(-1)
    
    intersection = (pred_binary * target).sum()
    return np.round((2. * intersection + epsilon) / (pred_binary.sum() + target.sum() + epsilon),6)

## CONCEPT

**SAMPLE TARGET**

In [109]:
target = torch.LongTensor([
    [1,1,1,1],
    [1,1,1,1],
    [0,0,0,0],
    [0,0,0,0]
])
target = target.unsqueeze(0) # add batch dimension
print(target.shape)

torch.Size([1, 4, 4])


**TEST 1 - 2 TENSORS ARE IDENTICAL -- EXPECTED DICE VALUE 1**

In [110]:
# fully overlap - we expect DICE=1
pred = torch.LongTensor([
    [1,1,1,1],
    [1,1,1,1],
    [0,0,0,0],
    [0,0,0,0]
])
pred = pred.unsqueeze(0) 
dice_score_manual_identical = dice_coefficient(pred, target)
print(dice_score_manual_identical)
dice_score_lib_identical = dice(pred, target, average='micro')
print(dice_score_lib_identical)

tensor(1.)
tensor(1.)


**TEST 2 - 2 TENSORS ARE COMPLETELY DIFFERENT -- EXPECTED DICE VALUE 0**

In [111]:
pred = torch.LongTensor([
    [0,0,0,0],
    [0,0,0,0],
    [1,1,1,1],
    [1,1,1,1],
])
pred = pred.unsqueeze(0) # add batch dimension
dice_score_dif_manual = dice_coefficient(pred, target)
print(dice_score_dif_manual)

dice_score_dif_lib = dice(pred, target, average='micro')
print(dice_score_dif_lib)

tensor(0.)
tensor(0.)


**TEST 4 - 2 TENSORS ARE 50% OVERLAPP -- EXPECTED DICE VALUE 0.5**

In [112]:
pred = torch.LongTensor([
    [1,1,0,0],
    [1,1,0,0],
    [1,1,0,0],
    [1,1,0,0]
])
pred = pred.unsqueeze(0) # add batch dimension
dice_score_half_manual = dice_coefficient(pred, target)
print(dice_score_half_manual)

dice_score_half_lib = dice(pred, target, average='micro')
print(dice_score_half_lib)

tensor(0.5000)
tensor(0.5000)


Summary: all tests for the function work correctly for manual implementation and for automated implementation from the library.

## TEST FUNCTIONS WITHOUT BINARIZATION

Check: is the part of code pred_binary = (pred > 0.5).float() necessary in the dice_coeff implementation?

Removing: the piece of code in the function and test how it works:

In [113]:
def dice_coefficient_changed(pred: torch.Tensor, target: torch.Tensor, epsilon: float = 1e-6) -> float:
    """
    Calculate the Dice coefficient between predictions and target.
    This version assumes that predictions are probabilities and targets are binary masks.
    """

    if pred.dim() == 4 and pred.shape[1] == 1:  
        pred = torch.softmax(pred,dim=1)
    else:
        pred = torch.sigmoid(pred)
    
    #pred_binary = (pred > 0.5).float() 
    #pred_binary = pred_binary.contiguous().view(-1)
    pred_binary = pred.contiguous().view(-1) #changed
    target = target.contiguous().view(-1)
    
    intersection = (pred_binary * target).sum()
    return np.round((2. * intersection + epsilon) / (pred_binary.sum() + target.sum() + epsilon),6)

In [114]:
# fully overlap - we expect DICE=1
pred = torch.LongTensor([
    [100,100,100,100],
    [100,100,100,100],
    [-100,-100,-100,-100],
    [-100,-100,-100,-100]
])
pred = pred.unsqueeze(0) # add batch dimension
dice_score1 = dice_coefficient_changed(pred, target)
print(dice_score1)
# fully not overlap - we expect DICE=0
pred = torch.LongTensor([
    [-100,-100,-100,-100],
    [-100,-100,-100,-100],
    [100,100,100,100],
    [100,100,100,100],
])
pred = pred.unsqueeze(0) # add batch dimension
dice_score_2= dice_coefficient_changed(pred, target)
print(dice_score_2)
# half overlap - we expect DICE=0.5
pred = torch.LongTensor([
    [100,100,-100,-100],
    [100,100,-100,-100],
    [100,100,-100,-100],
    [100,100,-100,-100]
])
pred = pred.unsqueeze(0) # add batch dimension
dice_score3= dice_coefficient_changed(pred, target)
print(dice_score3)
# random prediction: - we expect DICE=0.5
pred = torch.LongTensor([
    [0,0,-0,-0],
    [0,0,-0,-0],
    [0,0,-0,-0],
    [0,0,-0,-0]
])
pred = pred.unsqueeze(0) # add batch dimension
dice_score7 = dice_coefficient_changed(pred, target)
print(dice_score7)

tensor(1.)
tensor(0.)
tensor(0.5000)
tensor(0.5000)


Conclusion: the piece of code for binarisation does not impact the correect result of dice function.

## TESTING THE DICE FUNCTION ON THE SHAPES EXISITNG IN THE DATASET

- Pred mask shape: torch.Size([8, 128, 128])
- True mask shape: torch.Size([8, 1, 128, 128])


**GENERATING PRED_MASK AND TRUE_MASK**

In [115]:
torch.manual_seed(42)
pred_mask = torch.rand(8, 128, 128) #value probability float between 0 and 1
true_mask = torch.randint(0, 2, (8, 1, 128, 128))  # Values  0 or 1

print("Pred mask shape:", pred_mask.shape)
print("True mask shape:", true_mask.shape)

Pred mask shape: torch.Size([8, 128, 128])
True mask shape: torch.Size([8, 1, 128, 128])


In [116]:
print(pred_mask)

tensor([[[0.8823, 0.9150, 0.3829,  ..., 0.2083, 0.3289, 0.1054],
         [0.9192, 0.4008, 0.9302,  ..., 0.2576, 0.3470, 0.0240],
         [0.7797, 0.1519, 0.7513,  ..., 0.1474, 0.6872, 0.9231],
         ...,
         [0.3886, 0.6950, 0.9444,  ..., 0.5623, 0.4582, 0.1418],
         [0.2072, 0.2025, 0.8032,  ..., 0.1501, 0.0055, 0.7553],
         [0.0428, 0.2748, 0.8039,  ..., 0.6069, 0.1175, 0.9703]],

        [[0.7622, 0.7843, 0.0643,  ..., 0.9987, 0.1228, 0.4648],
         [0.4646, 0.2497, 0.1435,  ..., 0.1612, 0.7156, 0.9816],
         [0.5184, 0.2707, 0.2327,  ..., 0.0707, 0.2499, 0.3952],
         ...,
         [0.5721, 0.4261, 0.7520,  ..., 0.4272, 0.0380, 0.6681],
         [0.5586, 0.1097, 0.3872,  ..., 0.2815, 0.1916, 0.3653],
         [0.2675, 0.9510, 0.1335,  ..., 0.1357, 0.1557, 0.9772]],

        [[0.5288, 0.0632, 0.6178,  ..., 0.5601, 0.5215, 0.9896],
         [0.1072, 0.6026, 0.6029,  ..., 0.3892, 0.7201, 0.8710],
         [0.0533, 0.5834, 0.5451,  ..., 0.6317, 0.3324, 0.

In [117]:
print(true_mask)


tensor([[[[1, 0, 1,  ..., 1, 1, 1],
          [0, 1, 0,  ..., 0, 0, 0],
          [0, 1, 0,  ..., 0, 0, 1],
          ...,
          [1, 0, 1,  ..., 1, 0, 0],
          [1, 0, 1,  ..., 0, 0, 1],
          [1, 1, 0,  ..., 1, 0, 1]]],


        [[[1, 1, 0,  ..., 1, 1, 0],
          [1, 0, 1,  ..., 0, 0, 1],
          [1, 1, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 1, 0],
          [0, 1, 1,  ..., 0, 0, 1]]],


        [[[0, 0, 1,  ..., 0, 0, 0],
          [1, 1, 0,  ..., 0, 0, 1],
          [0, 0, 1,  ..., 1, 0, 0],
          ...,
          [0, 1, 0,  ..., 0, 0, 1],
          [1, 0, 1,  ..., 1, 1, 1],
          [0, 0, 1,  ..., 1, 1, 1]]],


        ...,


        [[[1, 1, 1,  ..., 1, 1, 0],
          [1, 1, 0,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 0, 1, 0],
          ...,
          [0, 1, 0,  ..., 1, 0, 1],
          [0, 1, 0,  ..., 1, 0, 1],
          [0, 1, 0,  ..., 0, 0, 1]]],


        [[[1, 1, 1,  ..., 0, 0, 0],
         

In [118]:
dice_score4 = dice_coefficient(pred_mask, true_mask)
print(dice_score4)

tensor(0.6664)


In [119]:
dice_score5 = dice_coefficient_changed(pred_mask, true_mask)
print(dice_score5)

tensor(0.5533)


In [120]:
dice_score6= dice(pred_mask, true_mask, average='micro')
print(dice_score6)

tensor(0.4994)
